## Convolutional Neural Network

### Libraries

In [1]:
# standart data tools
import numpy as np
import pandas as pd

# common visualizing tools
import matplotlib.pyplot as plt
import seaborn as sns

# CNN layers and the Deep Learning model
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense , Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from kerastuner.tuners import RandomSearch
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

# splitting tool for the validation set
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.utils import shuffle
print("GPUS: ", tf.config.list_physical_devices('GPU'))

import os
import cv2
from tqdm import tqdm

2024-06-16 16:57:21.795409: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-16 16:57:22.861709: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_39244/782462465.py:16: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


GPUS:  []


2024-06-16 16:57:23.976595: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-16 16:57:24.144914: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Label and load the dataset¶

In [2]:
train_dir = './seg_train'
test_dir = './seg_test'

## Assign class labels

In [3]:
class_names = ['buildings','forest','glacier','mountain','sea','street']
class_labels = {class_name:i for i, class_name in enumerate(class_names)}
print(class_labels)

number_classes = len(class_names)

IMAGE_SIZE = (150,150)

{'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}


## Load Dataset Function

In [4]:
# define load_dataset function to load dataset with labels
def load_dataset():
    
    '''
        Load the data:
            -14,034 images for training the model
            -3,000 images to evaluate how accurately the model learned to classify images
    '''
    
    # create list of datasets
    datasets = [train_dir, test_dir]
    output = []
    
    for dataset in datasets:
        
        images1 = []
        labels1 = []
        print(f"loading {dataset}")
        
        for folder in os.listdir(dataset):
            # assign labels to each folder images
            label = class_labels[folder]
            for file in tqdm(os.listdir(os.path.join(dataset,folder))):
                image_path = os.path.join(os.path.join(dataset, folder), file)
                # read the image files stored in image_path
                image_file = cv2.imread(image_path)
                image_file = cv2.cvtColor(image_file, cv2.COLOR_BGR2RGB)
                image_file = cv2.resize(image_file, IMAGE_SIZE)
                
                images1.append(image_file)
                labels1.append(label)
                
        # convert the images and labels list to numpy array
        images1 = np.array(images1, dtype = 'float32')
        labels1 = np.array(labels1, dtype = 'int32')
        
        output.append((images1, labels1))
        print("Images file have been loaded")
                
    return output

In [5]:
# load the data using above functions
((train_images, train_labels), (test_images, test_labels)) = load_dataset()

loading ./seg_train


100%|██████████| 2404/2404 [00:01<00:00, 2293.94it/s]


Images file have been loaded
loading ./seg_test


100%|██████████| 553/553 [00:00<00:00, 2729.08it/s]


Images file have been loaded


In [6]:
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)

In [7]:
# normalize the images dataset
train_images = train_images/255.0
test_images = test_images/255.0

### Creating the CNN Model

In [8]:
CNN_model = Sequential()

CNN_model.add(Conv2D(filters=16,kernel_size=(5,5),activation="relu",padding="same", input_shape = (150, 150, 3)))
CNN_model.add(Conv2D(filters=32,kernel_size=(5,5),activation="relu",padding="same"))
CNN_model.add(MaxPooling2D(pool_size=(2,2),padding="same"))

CNN_model.add(Conv2D(filters=32,kernel_size=(4,4),activation="relu",padding="same"))
CNN_model.add(Conv2D(filters=32,kernel_size=(4,4),activation="relu",padding="same"))
CNN_model.add(MaxPooling2D(pool_size=(2,2),padding="same"))

CNN_model.add(Dropout(0.2)) 

CNN_model.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu",padding="same"))
CNN_model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu",padding="same"))
CNN_model.add(MaxPooling2D(pool_size=(2,2),padding="same"))

CNN_model.add(Dropout(0.2))

CNN_model.add(Conv2D(filters=32,kernel_size=(2,2),activation="relu",padding="same"))
CNN_model.add(MaxPooling2D(pool_size=(2,2),padding="same"))

CNN_model.add(Dropout(0.2))

CNN_model.add(Flatten())

CNN_model.add(Dense(128,activation="relu"))
CNN_model.add(Dense(64,activation="relu"))
CNN_model.add(Dense(40,activation="relu"))

CNN_model.add(Dense(6,activation="softmax"))

# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), 
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation=tf.nn.relu),
#     tf.keras.layers.Dense(6, activation=tf.nn.softmax)
# ])

/home/jakerosa/Desktop/Uni/3ano_2semestre/TAA/P2_ML2024_108215_109089/env/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


### Summary of the Model

In [9]:
CNN_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 150, 150, 16)   │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 150, 150, 32)   │        12,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 75, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 75, 75, 32)     │        16,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 75, 75, 32)     │        16,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 38, 38, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 38, 38, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 38, 38, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 38, 38, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 19, 19, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 19, 19, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 19, 19, 32)     │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 10, 10, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 10, 10, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       409,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 40)             │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           246 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 503,678 (1.92 MB)

 Trainable params: 503,678 (1.92 MB)

 Non-trainable params: 0 (0.00 B)

### Compilation

In [10]:
CNN_model.compile(optimizer=Adam(learning_rate=0.0002),loss=keras.losses.categorical_crossentropy,metrics=["accuracy"])

# model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [11]:
train_labels = to_categorical(train_labels)

history = CNN_model.fit(train_images, train_labels, epochs=70, validation_split = 0.2)

# history = model.fit(train_images, train_labels, batch_size=128, epochs=20, validation_split = 0.2)

2024-06-16 16:57:43.464381: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3031290000 exceeds 10% of free system memory.


Epoch 1/70
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 962ms/step - accuracy: 0.3697 - loss: 1.4762

2024-06-16 17:03:30.264214: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 757890000 exceeds 10% of free system memory.


351/351 ━━━━━━━━━━━━━━━━━━━━ 353s 999ms/step - accuracy: 0.3701 - loss: 1.4755 - val_accuracy: 0.6088 - val_loss: 0.9929
Epoch 2/70
351/351 ━━━━━━━━━━━━━━━━━━━━ 373s 1s/step - accuracy: 0.6135 - loss: 0.9791 - val_accuracy: 0.6373 - val_loss: 0.9360
Epoch 3/70
351/351 ━━━━━━━━━━━━━━━━━━━━ 366s 1s/step - accuracy: 0.6615 - loss: 0.8505 - val_accuracy: 0.6573 - val_loss: 0.8436
Epoch 4/70
351/351 ━━━━━━━━━━━━━━━━━━━━ 352s 1s/step - accuracy: 0.6972 - loss: 0.7793 - val_accuracy: 0.7104 - val_loss: 0.7727
Epoch 5/70
351/351 ━━━━━━━━━━━━━━━━━━━━ 359s 1s/step - accuracy: 0.7338 - loss: 0.6947 - val_accuracy: 0.7481 - val_loss: 0.6645
Epoch 6/70
351/351 ━━━━━━━━━━━━━━━━━━━━ 364s 1s/step - accuracy: 0.7766 - loss: 0.6165 - val_accuracy: 0.7834 - val_loss: 0.5923
Epoch 7/70
351/351 ━━━━━━━━━━━━━━━━━━━━ 364s 1s/step - accuracy: 0.7962 - loss: 0.5633 - val_accuracy: 0.8123 - val_loss: 0.5558
Epoch 8/70
351/351 ━━━━━━━━━━━━━━━━━━━━ 358s 1s/step - accuracy: 0.8123 - loss: 0.5187 - val_accuracy: 0.

### Plotting Training and Validating Accuracy/Loss

In [ ]:
plt.figure(figsize=(24,8))

plt.subplot(1,2,1)
plt.plot(history.history["val_accuracy"],label="validation_accuracy",c="red",linewidth=4)
plt.plot(history.history["accuracy"],label="training_accuracy",c="green",linewidth=4)
plt.legend()
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(history.history["val_loss"],label="validation_loss",c="red",linewidth=4)
plt.plot(history.history["loss"],label="training_loss",c="green",linewidth=4)
plt.legend()
plt.grid(True)

plt.suptitle("ACC / LOSS",fontsize=18)

plt.show()

### Confusion Matrix Function

In [ ]:
# Define train and analyse model functions
from sklearn.model_selection import GridSearchCV, KFold, validation_curve, cross_validate


def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

### Error Analysis

#### Confusion Matrix

In [ ]:
y_pred = CNN_model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(test_labels, axis=1)

datasets = {
    "Test": (y_test_classes, y_pred_classes)
}

for dataset, (y_true, y_prediction) in datasets.items():
    print(f"Results for {dataset} set")
    print("Accuracy: ", accuracy_score(y_true, y_prediction))
    print("F1 Score: ", f1_score(y_true, y_prediction, average='weighted'))
    print(classification_report(y_true, y_prediction))
    cm = confusion_matrix(y_true, y_prediction)
    plot_confusion_matrix(cm, target_names=np.unique(y_true).astype(str), title=f'{dataset} Set Confusion Matrix', normalize=False)


#### Examples of mislabeled images by the classifier

In [ ]:
# Função para plotar imagens mal classificadas
def plot_mislabeled_images(test_images, true_labels, pred_labels, class_names, num_images=25):
    """
    Plots examples of mislabeled images.
    
    Parameters:
    - test_images: array of test images.
    - true_labels: array of true labels.
    - pred_labels: array of predicted labels by the classifier.
    - class_names: list of class names.
    - num_images: number of mislabeled images to display (default is 25).
    """
    mislabeled_indices = np.where(true_labels != pred_labels)[0]
    num_mislabeled = len(mislabeled_indices)
    
    if num_mislabeled == 0:
        print("No mislabeled images found.")
        return
    
    # Select up to `num_images` mislabeled images
    num_images = min(num_images, num_mislabeled)
    mislabeled_indices = np.random.choice(mislabeled_indices, num_images, replace=False)
    
    plt.figure(figsize=(15, 15))
    
    for i, index in enumerate(mislabeled_indices):
        plt.subplot(5, 5, i + 1)
        plt.imshow(test_images[index])
        plt.title(f"True: {class_names[true_labels[index]]}\nPred: {class_names[pred_labels[index]]}")
        plt.axis('off')
    
    plt.suptitle('Examples of Mislabeled Images by the Classifier', size=20)
    plt.show()

plot_mislabeled_images(test_images, y_test_classes, y_pred_classes, class_names)

### Feature Extraction with VGG ImageNet Model